In [ ]:
# #Download and Extract the Dataset:
# !wget https://ai2-s2-mslr.s3.us-west-2.amazonaws.com/mslr_data.tar.gz
# !tar -xvf mslr_data.tar.gz

# #Delete the Cochrane dataset and any other unwanted files:
# !rm -r mslr_data/cochrane/
# !rm mslr_data.tar.gz*

# #Move the ms2 directory up one level and remove the parent mslr_data directory:
# !mv mslr_data/ms2 ./
# !rm -r mslr_data/
# !rm -r sample_data/

In [1]:
#loading the data through hugginface

import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("allenai/mslr2022", "ms2", split='validation')

# Convert the dataset to a Pandas DataFrame
df = dataset.to_pandas()

# Save the DataFrame to a CSV file
df.to_csv("mslr2022_validation.csv", index=False)

print("Dataset saved to mslr2022_validation.csv")


Generating train split:   0%|          | 0/14188 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1667 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2021 [00:00<?, ? examples/s]

Dataset saved to mslr2022_validation.csv


_______

In [4]:
# sure the GPUs are working 
import tensorflow as tf

# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

#import 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration, AutoConfig
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch

import evaluate
from pprint import pprint  # Makes longer output readable without horizontal scrolling

import csv





TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.9.0


In [7]:

# Load the data- test inputs
df_dev = pd.read_csv('mslr2022_validation.csv')


df_dev = df_dev[['review_id', 'abstract']]

df_dev.head()

,review_id,abstract
0,28514886,"[""ABSTRACT A healthy intestinal microbiota is ..."
1,18842808,['The effects of the soluble fiber konjac gluc...
2,24297836,['The aims of this study were 1 ) to evaluate ...
3,32367221,"[""Abstract . This study documented postoperati..."
4,25038833,['OBJECTIVES To investigate the effects of dar...


In [8]:
print(df_dev.iloc[0, 1])


["ABSTRACT A healthy intestinal microbiota is considered to be important for priming of the infants ' mucosal and systemic immunity . Breast-fed infants typically have an intestinal microbiota dominated by different Bifidobacterium species . It has been described that allergic infants have different levels of specific Bifidobacterium species than healthy infants . For the accurate quantification of Bifidobacterium adolescentis , Bifidobacterium angulatum , Bifidobacterium bifidum , Bifidobacterium breve , Bifidobacterium catenulatum , Bifidobacterium dentium , Bifidobacterium infantis , and Bifidobacterium longum in fecal sample s , duplex 5′ nuclease assays were developed . The assays , targeting rRNA gene intergenic spacer regions , were vali date d and compared with conventional PCR and fluorescent in situ hybridization methods . The 5′ nuclease assays were subsequently used to determine the relative amounts of different Bifidobacterium species in fecal sample s from infants receivi

In [ ]:
# # Replace 'your_review_id' with the actual ReviewID you're looking for
# your_review_id = 9712593

# # Select the row where 'ReviewID' matches 'your_review_id'
# target_text = df.loc[df['ReviewID'] == your_review_id, 'Abstract'].values[0]

# print(target_text)

_______

***Run the code on validation dataset***

In [14]:
# Initialize the Pegasus tokenizer and model
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-large")
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")

# Tokenize the inputs
inputs = tokenizer(df_dev['abstract'].tolist(), max_length=1024, 
                    truncation=True, padding="max_length", return_tensors="pt")  # Note return_tensors="pt" for PyTorch


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Define the batch size
batch_size = 10

# Initialize the results list
results = []

# Generate summaries in batches
for i in range(0, len(inputs['input_ids']), batch_size):
    input_ids_batch = inputs['input_ids'][i:i+batch_size]
    review_ids_batch = df_dev['review_id'][i:i+batch_size].tolist()  # Get the corresponding ReviewIDs
    summary_ids = model.generate(input_ids_batch,
                                 num_beams=2,
                                 no_repeat_ngram_size=2,
                                 min_length=10,
                                 max_length=512,
                                 early_stopping=True)
    batch_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    # Append each summary with its ReviewID to the results list
    for review_id, summary in zip(review_ids_batch, batch_summaries):
        results.append({'review_id': review_id, 'Summary': summary})

# Convert the results to a DataFrame
summaries_df_val = pd.DataFrame(results)


# Save the DataFrame to a CSV file
output_file_val = 'val-prediction.csv'
summaries_df_val.to_csv(output_file_val, index=True)
print(f"Saved summaries to {output_file_val}")

Saved summaries to val-prediction.csv
